In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

# Normalizar os dados e salvar o scaler
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(df_numeric_filled)
joblib.dump(scaler, 'scaler.joblib')

# Converter os dados para tensores do PyTorch
X_train, X_test, y_train, y_test = train_test_split(df_normalized, y_labels, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Definição do Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=10):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)  # Compressão para espaço latente
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)  # Reconstrução dos dados originais
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Criar modelo, definir otimizador e função de perda
input_dim = X_train.shape[1]
autoencoder = Autoencoder(input_dim, latent_dim=10)
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Treinar o Autoencoder
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = autoencoder(X_train_tensor)
    loss = criterion(output, X_train_tensor)  # Comparando reconstrução com entrada
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Época {epoch}, Loss: {loss.item():.4f}')

# Salvar o Autoencoder como um arquivo .pth
torch.save(autoencoder.state_dict(), 'autoencoder.pth')

# Extrair representações latentes
X_train_latent = autoencoder.encoder(X_train_tensor).detach().numpy()
X_test_latent = autoencoder.encoder(X_test_tensor).detach().numpy()

# Treinar um classificador RandomForest com os dados latentes
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_latent, y_train)

# Salvar o classificador RandomForest
joblib.dump(clf, 'random_forest.joblib')
